In [2]:
from tools.audio_recorder import AudioRecorder
from tools.audio_transcriber import AudioTranscriber
import keyboard

def listen_for_key():
    recorder = AudioRecorder()
    transcriber = AudioTranscriber()

    try:
        print("Press and hold the spacebar to record.")
        print("Release the spacebar to stop recording.")
        print("Press Esc to terminate the program.")

        while True:
            # When spacebar is pressed, start recording
            if keyboard.is_pressed('space'):
                if not recorder.recording:
                    recorder.start_recording()

            # When spacebar is released, stop recording
            elif recorder.recording:
                recorder.stop_recording()
                transcriber.transcribe_audio(recorder.OUTPUT_FILENAME)

            # If the Esc key is pressed, terminate the program
            if keyboard.is_pressed('esc'):
                if recorder.recording:
                    recorder.stop_recording()
                print("Terminating program...")
                break
    finally:
        recorder.cleanup()

if __name__ == "__main__":
    listen_for_key()


Press and hold the spacebar to record.
Release the spacebar to stop recording.
Press Esc to terminate the program.
Recording started...
Recording saved to ./recordings\output.wav
Transcribing ./recordings\output.wav...
Transcription completed.
 Testing, please record this audio please.
Terminating program...
